# Day 12 - o1-preview (GHC)

In [1]:
with open('input.txt', 'r') as f:
    grid = [list(line.strip()) for line in f if line.strip()]

rows = len(grid)
cols = len(grid[0]) if rows > 0 else 0
visited = [[False for _ in range(cols)] for _ in range(rows)]

total_price = 0

def dfs(r, c, plant_type, region_cells):
    stack = [(r, c)]
    visited[r][c] = True
    while stack:
        i, j = stack.pop()
        region_cells.append((i, j))
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
            ni, nj = i + dr, j + dc
            if 0 <= ni < rows and 0 <= nj < cols:
                if not visited[ni][nj] and grid[ni][nj] == plant_type:
                    visited[ni][nj] = True
                    stack.append((ni, nj))

for i in range(rows):
    for j in range(cols):
        if not visited[i][j]:
            region_cells = []
            plant_type = grid[i][j]
            dfs(i, j, plant_type, region_cells)
            area = len(region_cells)
            perimeter = 0
            for r, c in region_cells:
                sides = 4
                for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < rows and 0 <= nc < cols:
                        if grid[nr][nc] == plant_type:
                            sides -= 1
                perimeter += sides
            price = area * perimeter
            total_price += price

print(total_price)

1449902


## Part 2

In [7]:
with open('input.txt', 'r') as f:
    grid = [list(line.strip()) for line in f if line.strip()]

rows = len(grid)
cols = len(grid[0]) if rows > 0 else 0

visited = [[False]*cols for _ in range(rows)]
total_price = 0

def is_boundary(r, c, plant_type):
    for dr, dc in [(-1,0), (0,1), (1,0), (0,-1)]:
        nr, nc = r + dr, c + dc
        if not (0 <= nr < rows and 0 <= nc < cols) or grid[nr][nc] != plant_type:
            return True
    return False

def dfs(r, c, plant_type):
    stack = [(r, c)]
    region_cells = set()
    visited[r][c] = True
    while stack:
        i, j = stack.pop()
        region_cells.add((i, j))
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
            ni, nj = i + dr, j + dc
            if 0 <= ni < rows and 0 <= nj < cols:
                if not visited[ni][nj] and grid[ni][nj] == plant_type:
                    visited[ni][nj] = True
                    stack.append((ni, nj))
    return region_cells

def trace_boundary(region_cells, plant_type):
    # Find boundary cells
    boundary_cells = []
    for r, c in region_cells:
        if is_boundary(r, c, plant_type):
            boundary_cells.append((r, c))

    # Start from the smallest boundary cell coordinate
    start_cell = min(boundary_cells)
    current_cell = start_cell
    boundary = []
    chain_code = []
    directions = [(-1,0), (0,1), (1,0), (0,-1)]  # N, E, S, W
    dir_indices = {(-1,0):0, (0,1):1, (1,0):2, (0,-1):3}
    previous_direction = None
    visited_boundary = set()

    while True:
        if (current_cell, previous_direction) in visited_boundary:
            break
        visited_boundary.add((current_cell, previous_direction))
        for idx, (dr, dc) in enumerate(directions):
            nr, nc = current_cell[0] + dr, current_cell[1] + dc
            if not (0 <= nr < rows and 0 <= nc < cols) or grid[nr][nc] != plant_type:
                continue
            if is_boundary(nr, nc, plant_type):
                if previous_direction != idx:
                    chain_code.append(idx)
                previous_direction = idx
                current_cell = (nr, nc)
                break
        else:
            break
        if current_cell == start_cell and previous_direction == chain_code[0]:
            break

    if not chain_code:
        sides = 4  # Single cell region
    else:
        sides = 1
        for i in range(1, len(chain_code)):
            if chain_code[i] != chain_code[i-1]:
                sides += 1
    return sides

for i in range(rows):
    for j in range(cols):
        if not visited[i][j]:
            plant_type = grid[i][j]
            region_cells = dfs(i, j, plant_type)
            area = len(region_cells)
            sides = trace_boundary(region_cells, plant_type)
            price = area * sides
            total_price += price

print(total_price)

66401
